In [1]:
import modin.pandas as pd
from distributed import Client

# Inicializa un cliente Dask para la ejecución distribuida en múltiples núcleos
client = Client(memory_limit='8GB')

# Lista de nombres de archivos
files = ["fhvhv_tripdata_2020-01.parquet", "fhvhv_tripdata_2020-02.parquet", 
         "fhvhv_tripdata_2020-03.parquet", "fhvhv_tripdata_2020-04.parquet",
         "fhvhv_tripdata_2020-05.parquet", "fhvhv_tripdata_2020-06.parquet",
         "fhvhv_tripdata_2020-07.parquet", "fhvhv_tripdata_2020-08.parquet",
         "fhvhv_tripdata_2020-09.parquet", "fhvhv_tripdata_2020-10.parquet",
         "fhvhv_tripdata_2020-11.parquet", "fhvhv_tripdata_2020-12.parquet"]

# Inicializa un DataFrame vacío con Modin
df_concatenado = pd.DataFrame()

# Lee y concatena cada archivo
for file in files:
    path = "../raw_data/" + file
    df_temporal = pd.read_parquet(path)
    tlc = pd.concat([df_concatenado, df_temporal], axis=0)

# Restablece los índices después de la concatenación
tlc.reset_index(drop=True, inplace=True)

# Imprime el DataFrame resultante
tlc.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B02764,B02764,2020-12-01 00:08:29,2020-12-01 00:11:29,2020-12-01 00:13:22,2020-12-01 00:33:53,94,75,6.90,...,1.79,0.00,NaN,0.00,17.91,N,N,,N,N
1,HV0003,B02764,B02764,2020-12-01 00:44:34,2020-12-01 00:47:00,2020-12-01 00:47:19,2020-12-01 00:57:01,75,164,3.73,...,1.41,2.75,NaN,1.88,8.99,N,N,,N,N
2,HV0005,B02510,None,2020-12-01 00:10:54,NaT,2020-12-01 00:17:14,2020-12-01 00:37:42,87,179,11.73,...,2.58,2.75,NaN,0.00,23.30,N,N,N,N,N
3,HV0003,B02883,B02883,2020-11-30 23:58:37,2020-12-01 00:00:01,2020-12-01 00:01:16,2020-12-01 00:20:26,80,92,8.34,...,1.77,0.00,NaN,2.82,18.82,N,N,,N,N
4,HV0003,B02883,B02883,2020-12-01 00:25:44,2020-12-01 00:30:41,2020-12-01 00:32:03,2020-12-01 00:45:18,92,92,2.77,...,1.00,0.00,NaN,0.00,9.70,N,N,,N,N


In [2]:
tlc.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 11637123 entries, 0 to 11637122
Data columns (total 24 columns):
 #   Column                Dtype         
---  ------                -----         
 0   hvfhs_license_num     object        
 1   dispatching_base_num  object        
 2   originating_base_num  object        
 3   request_datetime      datetime64[ns]
 4   on_scene_datetime     datetime64[ns]
 5   pickup_datetime       datetime64[ns]
 6   dropoff_datetime      datetime64[ns]
 7   PULocationID          int64         
 8   DOLocationID          int64         
 9   trip_miles            float64       
 10  trip_time             int64         
 11  base_passenger_fare   float64       
 12  tolls                 float64       
 13  bcf                   float64       
 14  sales_tax             float64       
 15  congestion_surcharge  float64       
 16  airport_fee           float64       
 17  tips                  float64       
 18  driver_pay            float64      

In [3]:
tlc.describe()

,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay
count,11637114,8491503,11637123,11637123,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.163712e+07,1.0,1.163712e+07,1.163712e+07
mean,2020-12-16 12:03:59.066150400,2020-12-16 08:07:01.561928192,2020-12-16 12:08:31.822571520,2020-12-16 12:25:23.758414336,1.348970e+02,1.383563e+02,4.503453e+00,1.011942e+03,1.765629e+01,7.413510e-01,5.599797e-01,1.517027e+00,8.494166e-01,0.0,5.102277e-01,1.437211e+01
min,2020-11-29 21:18:22,2020-11-30 23:56:06,2020-12-01 00:00:00,2020-12-01 00:02:45,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,-1.158500e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,-1.478000e+01
25%,2020-12-08 20:23:14,2020-12-08 18:13:24,2020-12-08 20:27:06,2020-12-08 20:41:33,6.900000e+01,7.100000e+01,1.550000e+00,5.480000e+02,8.900000e+00,0.000000e+00,2.700000e-01,7.700000e-01,0.000000e+00,0.0,0.000000e+00,6.820000e+00
50%,2020-12-16 12:05:46,2020-12-16 08:05:08,2020-12-16 12:09:46,2020-12-16 12:26:32,1.360000e+02,1.380000e+02,2.818000e+00,8.440000e+02,1.362000e+01,0.000000e+00,4.100000e-01,1.170000e+00,0.000000e+00,0.0,0.000000e+00,1.095000e+01
75%,2020-12-23 20:19:57.750000128,2020-12-23 16:21:32,2020-12-23 20:23:52,2020-12-23 20:42:21,2.080000e+02,2.130000e+02,5.540000e+00,1.285000e+03,2.175000e+01,0.000000e+00,6.800000e-01,1.890000e+00,2.750000e+00,0.0,0.000000e+00,1.784000e+01
max,2021-01-01 00:05:00,2020-12-31 23:59:41,2020-12-31 23:59:59,2021-01-01 04:46:08,2.650000e+02,2.650000e+02,4.003600e+02,5.221900e+04,2.233310e+03,9.618000e+01,4.408000e+01,9.911000e+01,1.100000e+01,0.0,4.700000e+02,1.607980e+03
std,NaN,NaN,NaN,NaN,7.667867e+01,7.847969e+01,5.032814e+00,6.828166e+02,1.383917e+01,3.110066e+00,4.967758e-01,1.186424e+00,1.277840e+00,NaN,1.855819e+00,1.155811e+01


In [5]:
tlc.isna().sum()

hvfhs_license_num              0
dispatching_base_num           0
originating_base_num     3154264
request_datetime               9
on_scene_datetime        3145620
pickup_datetime                0
dropoff_datetime               0
PULocationID                   0
DOLocationID                   0
trip_miles                     0
trip_time                      0
base_passenger_fare            0
tolls                          0
bcf                            0
sales_tax                      0
congestion_surcharge           0
airport_fee             11637122
tips                           0
driver_pay                     0
shared_request_flag            0
shared_match_flag              0
access_a_ride_flag             0
wav_request_flag               0
wav_match_flag                 0
dtype: int64

In [6]:
tlc.shape

(11637123, 24)

In [7]:
tlc.hvfhs_license_num.unique()

array(['HV0003', 'HV0005', 'HV0004'], dtype=object)